In [ ]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient

import requests, json
import pandas as pd

In [ ]:
# Carrega as variáveis do arquivo .env
load_dotenv()
database_url = os.getenv('DATABASE_URL')

# Faz conexão com Atlas
cliente = MongoClient(database_url)
db = cliente['projeto_fecomercio']

In [ ]:
# Endepoints
url_desemprego = "https://servicodados.ibge.gov.br/api/v3/agregados/4099/periodos/202001-202301/variaveis/4099?localidades=N3[all]"
url_inpc = "https://servicodados.ibge.gov.br/api/v3/agregados/7063/periodos/202001-202407/variaveis/44|68?localidades=N1[all]&classificacao=315[all]"
url_pmc = "https://servicodados.ibge.gov.br/api/v3/agregados/8883/periodos/202001-202410/variaveis/7169|11708|11710?localidades=N1[all]&classificacao=11046[56735]|85[all]"
url_pms = "https://servicodados.ibge.gov.br/api/v3/agregados/8163/periodos/202001-202410/variaveis/7167|11623|11625?localidades=N1[all]&classificacao=11046[56725]|1274[all]"
url_pim = "https://servicodados.ibge.gov.br/api/v3/agregados/8885/periodos/202001-202410/variaveis/12606|11602?localidades=N1[all]&classificacao=542[all]"


In [ ]:
# Realizar a requisição
response_desemprego = requests.get(url_desemprego)
print(f'desemprego: {response_desemprego.status_code}')
response_inpc = requests.get(url_inpc)
print(f'inpc: {response_inpc.status_code}')
response_pmc = requests.get(url_pmc)
print(f'pmc: {response_pmc.status_code}')
response_pms = requests.get(url_pms)
print(f'pms: {response_pms.status_code}')
response_pim = requests.get(url_pim)
print(f'pim: {response_pim.status_code}')

### desemprego

In [ ]:
# Verificar se a requisição de desemprego foi bem-sucedida
if response_desemprego.status_code == 200:
    data = response_desemprego.json()
    print(data)
    # Extrair e organizar os dados
    desemprego_data = []
    for resultado in data:
        for serie in resultado["resultados"][0]["series"]:
            estado_nome = serie["localidade"]["nome"]  # Obtém o nome do estado
            for periodo, valor in serie["serie"].items():
                desemprego_data.append({
                    "ano": periodo[:4],
                    "mes": periodo[4:],
                    "desemprego": valor,
                    "estado": estado_nome
                })
                
    df_desemprego = pd.DataFrame(desemprego_data)
else:
    print("Erro:", response_desemprego.status_code, response_desemprego.text)

In [ ]:
collection_desemprego = db['desemprego']
collection_desemprego.insert_many(df_desemprego.to_dict('records'))

### inpc

In [ ]:
# Função responsável por transformar os dados da api em df
def process_ibge_json(data):
    # Inicializa as variáveis de hierarquia
    current_group = None
    current_subgrupo = None
    current_item = None
    rows = []

    # Itera sobre cada resultado no JSON
    for resultado in data[0]['resultados']:
        classificacoes = resultado['classificacoes']
        for classificacao in classificacoes:
            for codigo, descricao in classificacao['categoria'].items():
                # Verifica se a descrição contém '.' para hierarquia
                if '.' in descricao:
                    partes = descricao.split('.', 1)
                    prefix = partes[0]
                    nome = partes[1]

                    # Determina o nível hierárquico baseado no comprimento do prefixo
                    if len(prefix) == 1:
                        # Grupo
                        current_group = nome
                    elif len(prefix) == 2:
                        # Subgrupo
                        current_subgrupo = nome
                    elif len(prefix) == 4:
                        # Item
                        current_item = nome
                    elif len(prefix) == 7:
                        # Subitem
                        subitem = nome
                        # Agora, processa a série correspondente
                        for serie in resultado['series']:
                            localidade = serie['localidade']['nome']
                            for periodo, valor in serie['serie'].items():
                                ano = periodo[:4]
                                mes = periodo[4:]
                                try:
                                    valor_float = float(valor)
                                except ValueError:
                                    valor_float = None  # Tratar valores inválidos

                                rows.append({
                                    "Grupo": current_group,
                                    "Subgrupo": current_subgrupo,
                                    "Item": current_item,
                                    "Subitem": subitem,
                                    "Ano": ano,
                                    "Mês": mes,
                                    "Localidade": localidade,
                                    "Valor INPC": valor_float
                                })
                else:
                    # Ignora classificações sem hierarquia (como "Índice geral")
                    continue

    # Cria o DataFrame
    df = pd.DataFrame(rows)

    return df

In [ ]:
# Verificar se a requisição de pmc foi bem-sucedida
if response_inpc.status_code == 200:
    inpc_data = response_inpc.json()
    print(inpc_data)
    # Extrair e organizar os dados
    # Lista para armazenar linhas do DataFrame
    # Lista para armazenar os registros formatados

    df_inpc = process_ibge_json(inpc_data)
else:
    print("Erro:", response_inpc.status_code, response_inpc.text)

In [ ]:
collection_inpc = db['inpc']
collection_inpc.insert_many(df_inpc.to_dict('records'))

### pmc

In [ ]:
# Verificar se a requisição de pmc foi bem-sucedida
if response_pmc.status_code == 200:
    data = response_pmc.json()
    print(data)
    # Extrair e organizar os dados
    pmc_data = []

    for resultados in data:
        variavel = resultados["variavel"]
        for resultado in resultados["resultados"]:
            for k, v in resultado["classificacoes"][1]["categoria"].items():
                categoria = v
            for serie in resultado["series"]:
                for periodo, valor in serie["serie"].items():
                    pmc_data.append({
                        "variavel": variavel,
                        "categoria": categoria,
                        "ano": periodo[:4],
                        "mes": periodo[4:],
                        "valor": valor
                    })
    df_pmc = pd.DataFrame(pmc_data)
else:
    print("Erro:", response_pmc.status_code, response_pmc.text)

In [ ]:
collection_pmc = db['pmc']
collection_pmc.insert_many(df_pmc.to_dict('records'))

### pms

In [ ]:
# Verificar se a requisição de pms foi bem-sucedida
if response_pms.status_code == 200:
    data = response_pms.json()
    print(data)
    # Extrair e organizar os dados
    pms_data = []

    for resultados in data:
        variavel = resultados["variavel"]
        for resultado in resultados["resultados"]:
            for k, v in resultado["classificacoes"][1]["categoria"].items():
                categoria = v
            for serie in resultado["series"]:
                for periodo, valor in serie["serie"].items():
                    pms_data.append({
                        "variavel": variavel,
                        "categoria": categoria,
                        "mes": periodo[4:],
                        "ano": periodo[:4],
                        "valor": valor,
                    })
    df_pms = pd.DataFrame(pms_data)
else:
    print("Erro:", response_pms.status_code, response_pms.text)

In [ ]:
collection_pms = db['pms']
collection_pms.insert_many(df_pms.to_dict('records'))

### pim

In [ ]:
# Verificar se a requisição de pim foi bem-sucedida
if response_pim.status_code == 200:
    data = response_pim.json()
    print(data)
    # Extrair e organizar os dados
    pim_data = []

    for resultados in data:
        variavel = resultados["variavel"]
        for resultado in resultados["resultados"]:
            for k, v in resultado["classificacoes"][0]["categoria"].items():
                categoria = v
            for serie in resultado["series"]:
                for periodo, valor in serie["serie"].items():
                    pim_data.append({
                        "variavel": variavel,
                        "categoria": categoria,
                        "mes": periodo[4:],
                        "ano": periodo[:4],
                        "valor": valor,
                    })

    df_pim = pd.DataFrame(pim_data)
else:
    print("Erro:", response_pim.status_code, response_pim.text)

In [ ]:
collection_pim = db['pim']
collection_pim.insert_many(df_pim.to_dict('records'))